In [1]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
import pandas as pd
from astropy.visualization import simple_norm
from astropy.wcs import WCS
from dust_extinction.averages import CT06_MWGC, RL85_MWGC, RRP89_MWGC, F11_MWGC, I05_MWAvg, CT06_MWLoc, G21_MWAvg
import regions
from regions import Regions
from astropy.nddata import Cutout2D
from astropy.table import Table, Column, MaskedColumn
from astropy import table


In [2]:
cd ..

/blue/adamginsburg/savannahgramze/cloudc/lactea-filament/lactea-filament


In [3]:
import catalog_query as cq
from jwst_plots import make_cat_use, JWSTCatalog
from decaps_plots import DecapsCatalog, make_decaps_use
import cutout_manager as cm

In [4]:
cd notebooks/

/blue/adamginsburg/savannahgramze/cloudc/lactea-filament/lactea-filament/notebooks


In [5]:
basepath = '/orange/adamginsburg/jwst/cloudc/'

# Open Catalogs

### Gaia Catalog

In [6]:
gaia = Table.read('/orange/adamginsburg/jwst/cloudc/catalogs/gaia_cloudc_fov.fits')
gaia['skycoord'] = SkyCoord(gaia['RA_ICRS'], gaia['DE_ICRS'])

In [7]:
gaia[:3]

RA_ICRS,DE_ICRS,Source,e_RA_ICRS,e_DE_ICRS,Plx,e_Plx,PM,pmRA,e_pmRA,pmDE,e_pmDE,RUWE,FG,e_FG,Gmag,FBP,e_FBP,BPmag,FRP,e_FRP,RPmag,BP-RP,RV,e_RV,Vbroad,GRVSmag,QSO,Gal,NSS,XPcont,XPsamp,RVS,EpochPh,EpochRV,MCMCGSP,MCMCMSC,And,Teff,logg,__Fe_H_,Dist,A0,HIP,PS1,SDSS13,SKYM2,TYC2,URAT1,AllWISE,APASS9,GSC23,RAVE5,_2MASS,RAVE6,RAJ2000,DEJ2000,skycoord
deg,deg,,mas,mas,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,mag,,,mag,,,mag,mag,km / s,km / s,km / s,mag,,,,,,,,,,,,K,log(cm.s**-2),,pc,mag,,,,,,,,,,,,,deg,deg,"deg,deg"
float64,float64,int64,float64,float64,float64,float32,float64,float64,float32,float64,float32,float64,float64,float32,float64,float32,float32,float64,float32,float32,float64,float64,float64,float32,float64,float64,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,float64,float64,float64,float64,float64,int32,int64,int64,int32,bytes12,bytes15,bytes19,int32,bytes10,bytes16,bytes17,bytes21,float64,float64,SkyCoord
266.59232196778,-28.65699254349,4057519204408840320,0.0143,0.0104,0.6996,0.0161,2.426,0.193,0.018,-2.418,0.010,0.898,270173.23367,60.95,12.108261,7.625e+04,65.32,13.132955,2.858e+05,138.9,11.107685,2.025269,--,--,--,--,0,0,0,1,1,0,0,0,0,1,0,--,--,--,--,--,--,--,--,165250898,--,--,--,27972032,S8DO000001,--,17462214-2839251,--,266.59232099061,-28.65698179710,"266.59232196778,-28.65699254349"
266.59463394035,-28.65819391215,4057519208706336000,0.5268,0.4079,1.2884,0.4948,9.231,-9.228,0.682,-0.222,0.392,1.013,198.14741,0.8149,19.944897,45.79,5.753,21.186491,237.5,7.939,18.808569,2.377922,--,--,--,--,0,0,0,0,0,0,0,0,0,0,0,--,--,--,--,--,--,73612665945790484,--,--,--,--,--,--,--,--,--,--,266.59468067968,-28.65819292619,"266.59463394035,-28.65819391215"
266.59214504061,-28.65796768739,4057519208711300992,0.4767,0.3276,0.9083,0.4624,7.823,4.657,0.705,-6.286,0.367,1.184,277.37931,2.851,19.579681,441.5,78.92,18.726192,1344,136.4,16.926788,1.799404,--,--,--,--,0,0,0,0,0,0,0,0,0,0,0,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,266.59212145483,-28.65793974970,"266.59214504061,-28.65796768739"


### Decaps Catalog

In [8]:
decaps_cat = make_decaps_use()
decaps = decaps_cat.catalog
decaps['skycoord'] = SkyCoord(ra=decaps['ra']*u.deg, dec=decaps['dec']*u.deg)

In [9]:
decaps[:3]

ra,dec,posstdev,ra_ok,dec_ok,posstdev_ok,ndet,ndet_ok,nmag,nmag_ok,mean,stdev,err,median,q25,q75,ndet_cflux,ndet_cflux_ok,nmag_cflux,nmag_cflux_ok,mean_cflux,stdev_cflux,err_cflux,median_cflux,q25_cflux,q75_cflux,maglimit,epochrange,epochrange_ok,epochmean,epochmean_ok,dnt_all,dqflags_all,fracflux_avg,rchi2_avg,qf_avg,prn_avg,prl_avg,prr_avg,pre_avg,cchi2_avg,snr_avg,dnt_any,dqflags_any,_id,obj_id,skycoord
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"deg,deg"
float64,float64,float32,float64,float64,float32,int16,int16,int16[6],int16[6],float32[6],float32[6],float32[6],float32[6],float32[6],float32[6],int16,int16,int16[6],int16[6],float32[6],float32[6],float32[6],float32[6],float32[6],float32[6],float32[6],float64,float64,float64,float64,int32[6],int64[6],float32[6],float32[6],float32[6],float32[6],float32[6],float32[6],float32[6],float32[6],float32[6],int32[6],int64[6],int64,int64,SkyCoord
266.2567415536595,-28.00088999267355,0.09644666,266.2567415536595,-28.00088999267355,0.09644666,13,13,0 .. 6,0 .. 6,0.0 .. 7.745599e-09,0.0 .. 7.103565e-10,0.0 .. 2.915216e-10,0.0 .. 7.937175e-09,0.0 .. 7.470702e-09,0.0 .. 8.207482e-09,13,13,0 .. 6,0 .. 6,0.0 .. 7.721063e-09,0.0 .. 6.764225e-10,0.0 .. 3.8397127e-10,0.0 .. 7.899806e-09,0.0 .. 7.4203994e-09,0.0 .. 8.253182e-09,0.0 .. 21.365091,722.8257134000014,722.8257134000014,58007.49174939846,58007.49174939846,0 .. 0,0 .. 0,0.0 .. 0.73797244,0.0 .. 0.99717367,0.0 .. 1.0,0.0 .. 0.0072425976,0.0 .. 0.15577787,0.0 .. 0.81948566,0.0 .. 0.017493742,0.0 .. 642.2981,0.0 .. 11.832285,0 .. 0,0 .. 0,4900760819563323363,5098919203116720807,"266.2567415536595,-28.00088999267355"
265.83302520143997,-28.117372138274987,0.10661936,265.83302520143997,-28.117372138274987,0.10661936,4,4,0 .. 1,0 .. 1,0.0 .. 4.5506e-09,0.0 .. 0.0,0.0 .. 5.9362093e-10,0.0 .. 4.5506e-09,0.0 .. 4.5506e-09,0.0 .. 4.5506e-09,4,4,0 .. 1,0 .. 1,0.0 .. 4.149399e-09,0.0 .. 0.0,0.0 .. 9.466654e-10,0.0 .. 4.149399e-09,0.0 .. 4.149399e-09,0.0 .. 4.149399e-09,0.0 .. 21.321514,7.946313680004096,7.946313680004096,57866.3707875,57866.3707875,0 .. 0,0 .. 0,0.0 .. 0.2327578,0.0 .. 0.9794993,0.0 .. 1.0,0.0 .. 1.42468e-07,0.0 .. 5.393787e-05,0.0 .. 0.9999417,0.0 .. 4.212124e-06,0.0 .. 306.5295,0.0 .. 7.685012,0 .. 0,0 .. 0,4900760819554427803,5062907998342584909,"265.83302520143997,-28.117372138274987"
265.83339203653765,-28.210620476794347,0.26000202,265.83339203653765,-28.210620476794347,0.26000202,8,8,0 .. 6,0 .. 6,0.0 .. 9.415713e-09,0.0 .. 3.4364176e-09,0.0 .. 3.088686e-10,0.0 .. 9.191808e-09,0.0 .. 7.0025616e-09,0.0 .. 1.3008107e-08,8,8,0 .. 6,0 .. 6,0.0 .. 7.2793895e-09,0.0 .. 2.4252607e-09,0.0 .. 4.371212e-10,0.0 .. 7.600535e-09,0.0 .. 5.475495e-09,0.0 .. 9.61777e-09,0.0 .. 21.252556,469.5820205800046,469.5820205800046,58042.22434520375,58042.22434520375,0 .. 3,0 .. 0,0.0 .. 0.2962493,0.0 .. 2.1593347,0.0 .. 1.0000001,0.0 .. 3.6349946e-05,0.0 .. 0.0021282826,0.0 .. 0.99700207,0.0 .. 0.00083336624,0.0 .. 405.53812,0.0 .. 13.035642,0 .. 3,0 .. 0,4900760819554427802,5062907998342584907,"265.83339203653765,-28.210620476794347"


### VVV Catalog

In [10]:
vvv = Table.read('/orange/adamginsburg/jwst/cloudc/catalogs/jw02221-o002_t001_nircam_clear-f405n-merged_vvvcat.fits')
vvv['skycoord'] = SkyCoord(vvv['RAJ2000'], vvv['DEJ2000'], frame='icrs')

In [11]:
vvv[:3]

RAJ2000,DEJ2000,iauname,mClass,Zmag3,e_Zmag3,Zperrbits,Ymag3,e_Ymag3,Yperrbits,Jmag3,e_Jmag3,Jperrbits,Hmag3,e_Hmag3,Hperrbits,Ksmag3,e_Ksmag3,Ksperrbits,RA,DEC,skycoord
deg,deg,,,mag,mag,,mag,mag,,mag,mag,,mag,mag,,mag,mag,,deg,deg,"deg,deg"
float64,float64,bytes24,int16,float32,float64,int32,float64,float64,int32,float64,float64,int32,float64,float64,int32,float64,float64,int32,float64,float64,SkyCoord
266.581129,-28.666482,VVV J174619.47-283959.33,-1,19.834,0.213,0,18.378,0.095,0,16.819,0.044,0,--,--,0,--,--,0,266.581129,-28.666482,"266.581129,-28.666482"
266.594517,-28.666434,VVV J174622.68-283959.16,-1,19.698,0.186,0,17.913,0.063,0,16.101,0.023,0,14.765,0.028,16,13.780,0.032,16,266.594517,-28.666434,"266.594517,-28.666434"
266.604305,-28.663877,VVV J174625.03-283949.95,1,19.802,0.203,0,19.139,0.195,0,18.163,0.148,0,--,--,0,--,--,0,266.604305,-28.663877,"266.604305,-28.663877"


### JWST

In [12]:
#cat_use = make_cat_use()

In [13]:
#cat_use.catalog[:3]

# Merge Catalogs

In [14]:
def match_crds(basecrds, tb, tb_name):
    max_offset = 0.1*u.arcsec
    crds = tb['skycoord']

    matches, sep, _ = crds.match_to_catalog_sky(basecrds, nthneighbor=1)
    reverse_matches, reverse_sep, _ = basecrds.match_to_catalog_sky(crds, nthneighbor=1)
    mutual_matches = (reverse_matches[matches] == np.arange(len(matches)))

    newcrds = crds[(sep > max_offset) | (~mutual_matches)]
    basecrds = SkyCoord([basecrds, newcrds])
    return basecrds

In [15]:
tbls = [gaia, decaps, vvv]
tbl_names = ['gaia', 'decaps', 'vvv']

ref_filter = 'gaia'
basecrds = gaia['skycoord']

max_offset = 0.1*u.arcsec

for i, tbl in enumerate(tbls):
    if tbl_names[i] == ref_filter:
        continue
    basecrds = match_crds(basecrds, tbl, tbl_names[i])

In [16]:
# Build the basetable
basetable = Table()
basetable['skycoord_ref'] = basecrds

for tbl, tbl_name in zip(tbls, tbl_names):
    wl = tbl_name

    crds = tbl['skycoord']
    matches, sep, _ = basecrds.match_to_catalog_sky(crds, nthneighbor=1)
    reverse_matches, reverse_sep, _ = crds.match_to_catalog_sky(basecrds, nthneighbor=1)
    mutual_matches = (reverse_matches[matches] == np.arange(len(matches)))

    basetable.add_column(name=f"sep_{wl}", col=sep)
    basetable.add_column(name=f'id_{wl}', col=matches)
    matchtb = tbl[matches]
    badsep = sep > max_offset

    for cn in matchtb.colnames:
        if isinstance(matchtb[cn], SkyCoord):
            matchtb.rename_column(cn, f"{cn}_{wl}")
            matchtb[f'mask_{wl}'] = badsep
        else:
            matchtb[f'{cn}_{wl}'] = MaskedColumn(data=matchtb[cn], name=f'{cn}_{wl}')
            matchtb[f'{cn}_{wl}'].mask[badsep] = True
            # mask non-mutual matches
            matchtb[f'{cn}_{wl}'].mask[~mutual_matches] = True
            matchtb.remove_column(cn)
    basetable = table.hstack([basetable, matchtb], join_type='exact')



In [ ]:
basetable.write('/orange/adamginsburg/jwst/cloudc/catalogs/matched_gaia_decaps_vvv.fits', overwrite=True)

In [ ]:
#crds = cat_use.catalog['skycoord_ref']
#matches, sep, _ = basecrds.match_to_catalog_sky(crds, nthneighbor=1)
#reverse_matches, reverse_sep, _ = crds.match_to_catalog_sky(basecrds, nthneighbor=1)
#mutual_matches = (reverse_matches[matches] == np.arange(len(matches)))
#
#matchtb = cat_use.catalog[matches]
#
#basetable = table.hstack([basetable, matchtb], join_type='exact')

: 